In [11]:
import pandas as pd
import geopandas as gpd
import numpy as np
from dateutil import parser
import matplotlib.pyplot as plt

import statistics

import json
from shapely.geometry import Point

import pickle

import time
import datetime
from datetime import datetime, timedelta

from mpl_toolkits.axes_grid1 import make_axes_locatable

In [2]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import f1_score


In [3]:
BIGMERGED_day = pd.read_pickle('../data/processed/BIGMERGED_day.pkl')
BIGMERGED_night = pd.read_pickle('../data/processed/BIGMERGED_night.pkl')

## REGRESSIONE

In [4]:
#parto dalla colonna day che già posseggo 

df_dayr = BIGMERGED_day[['day','weekday']]
df_nightr = BIGMERGED_night[['day','weekday']]

In [5]:
#tengo solo i day che sono compresi tra weekday 2 e 4
df_dayr = df_dayr[df_dayr['weekday'] != 0]
df_dayr = df_dayr[df_dayr['weekday'] != 1]
df_dayr = df_dayr.reset_index().drop('index', axis=1)

df_nightr = df_nightr[df_nightr['weekday'] != 0]
df_nightr = df_nightr[df_nightr['weekday'] != 1]
df_nightr = df_nightr.reset_index().drop('index', axis=1)

In [6]:
#ora voglio generare una lista con la stessa lunghezza, che al posto di day abbia day-1
dayr = df_dayr['day']
nightr = df_nightr['day']

In [12]:
#per df giorno

day1dr = [] #giorno precedente a day1
for i in range(len(dayr)):
    day1dr.insert(i, (datetime.strptime(dayr[i], '%Y-%m-%d') - timedelta(days=1)).strftime('%Y-%m-%d'))
     
day2dr = [] #giorno precedente a day2
for i in range(len(dayr)):
    day2dr.insert(i, (datetime.strptime(dayr[i], '%Y-%m-%d') - timedelta(days=1)).strftime('%Y-%m-%d'))
    

#per df notte 

day1nr = [] #giorno precedente a day1
for i in range(len(nightr)):
    day1nr.insert(i, (datetime.strptime(nightr[i], '%Y-%m-%d') - timedelta(days=1)).strftime('%Y-%m-%d'))
     
day2nr = [] #giorno precedente a day2
for i in range(len(dayr)):
    day2nr.insert(i, (datetime.strptime(nightr[i], '%Y-%m-%d') - timedelta(days=1)).strftime('%Y-%m-%d'))

In [13]:
daysr = pd.DataFrame({'day':dayr, 'day1':day1dr,'day2':day2dr})
#di questo df voglio tenere solo alcune righe che mi vadano bene. cioè voglio tenere le righe il cui primo giorno
#sia compreso tra mercoledi e venerdi (in modo da non sfociare nel weekend quando vado indietro di due giorni) -> FATTO SOPRA 
#poi i miei dati partono dal 28 novembre percui voglio eliminare le righe dove day2=27nov/26nov 

nightsr = pd.DataFrame({'day':dayr, 'day1':day1nr,'day2':day2nr})

In [14]:
daysr = daysr[daysr['day2'] != '2013-11-26'] 
daysr = daysr[daysr['day2'] != '2013-11-27']
daysr = daysr.groupby(['day','day1','day2']).sum().reset_index()

In [15]:
nightsr = nightsr[nightsr['day2'] != '2013-11-26'] 
nightsr = nightsr[nightsr['day2'] != '2013-11-27']
nightsr = nightsr.groupby(['day','day1','day2']).sum().reset_index()

In [16]:
BIGMERGED_day1 = BIGMERGED_day.merge(daysr, how='left', on='day')
BIGMERGED_day1 = BIGMERGED_day1[BIGMERGED_day1['day2'].notna()]
BIGMERGED_day1

,precipitation,minTemperature,maxTemperature,day,cellId,index_com,COMUNE,index_wstation,station,timestamp,weekday,num_tweets,consumo_ps,index_cons,day1,day2
993,0,-8.2,4.5,2013-11-29,4074.0,57.0,Grigno,34,T0450,1385679600,4.0,NaN,76.577328,1.0,2013-11-28,2013-11-28
994,0,-8.2,4.5,2013-11-29,4191.0,57.0,Grigno,34,T0450,1385679600,4.0,NaN,289.662068,1.0,2013-11-28,2013-11-28
995,0,-8.2,4.5,2013-11-29,4307.0,57.0,Grigno,34,T0450,1385679600,4.0,NaN,605.959729,2.0,2013-11-28,2013-11-28
996,0,-8.2,4.5,2013-11-29,4308.0,57.0,Grigno,34,T0450,1385679600,4.0,NaN,96.554023,1.0,2013-11-28,2013-11-28
997,0,-8.2,4.5,2013-11-29,4423.0,57.0,Grigno,34,T0450,1385679600,4.0,NaN,173.131351,1.0,2013-11-28,2013-11-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79355,1,11.8,15.8,2013-11-08,2147.0,14.0,Brentonico,24,T0401,1383865200,4.0,NaN,210.569855,1.0,2013-11-07,2013-11-07
79356,1,11.8,15.8,2013-11-08,2148.0,14.0,Brentonico,24,T0401,1383865200,4.0,NaN,63.404498,1.0,2013-11-07,2013-11-07
79357,1,11.8,15.8,2013-11-08,2261.0,14.0,Brentonico,24,T0401,1383865200,4.0,NaN,1.382455,1.0,2013-11-07,2013-11-07
79358,1,11.8,15.8,2013-11-08,2262.0,14.0,Brentonico,24,T0401,1383865200,4.0,NaN,7.051776,1.0,2013-11-07,2013-11-07


In [17]:
BIGMERGED_night1 = BIGMERGED_night.merge(nightsr, how='left', on='day')
BIGMERGED_night1 = BIGMERGED_night1[BIGMERGED_night1['day2'].notna()]
BIGMERGED_night1

,precipitation,minTemperature,maxTemperature,day,cellId,index_com,COMUNE,index_wstation,station,timestamp,weekday,num_tweets,consumo_ps,index_cons,day1,day2
993,0,-8.2,4.5,2013-11-29,4074.0,57.0,Grigno,34,T0450,1385679600,4.0,NaN,27.100023,1.0,2013-11-28,2013-11-28
994,0,-8.2,4.5,2013-11-29,4191.0,57.0,Grigno,34,T0450,1385679600,4.0,NaN,102.508784,1.0,2013-11-28,2013-11-28
995,0,-8.2,4.5,2013-11-29,4307.0,57.0,Grigno,34,T0450,1385679600,4.0,NaN,214.443664,1.0,2013-11-28,2013-11-28
996,0,-8.2,4.5,2013-11-29,4308.0,57.0,Grigno,34,T0450,1385679600,4.0,NaN,34.169595,1.0,2013-11-28,2013-11-28
997,0,-8.2,4.5,2013-11-29,4423.0,57.0,Grigno,34,T0450,1385679600,4.0,NaN,61.269618,1.0,2013-11-28,2013-11-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79355,1,11.8,15.8,2013-11-08,2147.0,14.0,Brentonico,24,T0401,1383865200,4.0,NaN,75.305624,1.0,2013-11-07,2013-11-07
79356,1,11.8,15.8,2013-11-08,2148.0,14.0,Brentonico,24,T0401,1383865200,4.0,NaN,20.871615,1.0,2013-11-07,2013-11-07
79357,1,11.8,15.8,2013-11-08,2261.0,14.0,Brentonico,24,T0401,1383865200,4.0,NaN,0.579142,1.0,2013-11-07,2013-11-07
79358,1,11.8,15.8,2013-11-08,2262.0,14.0,Brentonico,24,T0401,1383865200,4.0,NaN,2.776123,1.0,2013-11-07,2013-11-07


In [18]:
#ora voglio associare per ogni day1 i dati (sulla stessa riga) che lo riguardano

BIGMERGED_day2 = BIGMERGED_day1.merge(BIGMERGED_day, how='left', right_on=['day','cellId'], left_on=['day1','cellId'])

In [19]:
BIGMERGED_night2 = BIGMERGED_night1.merge(BIGMERGED_night, how='left', right_on=['day','cellId'], left_on=['day1','cellId'])

In [20]:
BIGMERGED_day3 = BIGMERGED_day2.merge(BIGMERGED_day, how='left', right_on=['day','cellId'], left_on=['day2','cellId'])

In [21]:
BIGMERGED_night3 = BIGMERGED_night2.merge(BIGMERGED_night, how='left', right_on=['day','cellId'], left_on=['day2','cellId'])

In [22]:
#ora droppo un tot di roba che non me serve miga 
#arrivata a questo punto della vita il contrassegno _x indica day, _y indica day1, *nulla* indica day2

BIGMERGED_day4 = BIGMERGED_day3.drop(['day1','day2','station_x','station_y','station','COMUNE_x','COMUNE_y','COMUNE','index_cons_x','index_cons_y','index_cons','day_x','day_y','day','timestamp_y','timestamp','index_com_x','index_com_y','index_com','index_wstation_x','index_wstation_y','index_wstation','weekday_y','weekday'],axis=1)

BIGMERGED_night4 = BIGMERGED_night3.drop(['day1','day2','station_x','station_y','station','COMUNE_x','COMUNE_y','COMUNE','index_cons_x','index_cons_y','index_cons','day_x','day_y','day','timestamp_y','timestamp','index_com_x','index_com_y','index_com','index_wstation_x','index_wstation_y','index_wstation','weekday_y','weekday'],axis=1)

In [23]:
#per ora droppo anche tweets
BIGMERGED_day4 = BIGMERGED_day4.drop(['num_tweets_x','num_tweets_y','num_tweets'], axis=1)

BIGMERGED_night4 = BIGMERGED_night4.drop(['num_tweets_x','num_tweets_y','num_tweets'], axis=1)
#BIGMERGED_night4 

In [24]:
featuresr = BIGMERGED_day4.columns
featuresr = featuresr[featuresr != 'consumo_ps_x'] #tolgo 'consumi_ps' che è il mio targhet
featuresr

Index(['precipitation_x', 'minTemperature_x', 'maxTemperature_x', 'cellId',
       'timestamp_x', 'weekday_x', 'precipitation_y', 'minTemperature_y',
       'maxTemperature_y', 'consumo_ps_y', 'precipitation', 'minTemperature',
       'maxTemperature', 'consumo_ps'],
      dtype='object')

In [25]:
trainr, testr = train_test_split(BIGMERGED_day4, test_size = 0.25)

In [26]:
from  sklearn.linear_model import LinearRegression
Lin_reg = LinearRegression(fit_intercept=True, normalize='deprecated', copy_X=True, n_jobs=None, positive=False)

#per adesso è tutto normale non ho cambiato niente
#standardizzo dati
scaler = StandardScaler()

In [27]:
pipe_lin = Pipeline([('scaler', scaler), ('Linreg', Lin_reg)])

In [28]:
param_gridr = {
        'Linreg__n_jobs': [1] 
    }

In [29]:
gridsearchr = GridSearchCV(pipe_lin, param_grid=param_gridr, cv=10)

In [30]:
pipe_lin.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'scaler', 'Linreg', 'scaler__copy', 'scaler__with_mean', 'scaler__with_std', 'Linreg__copy_X', 'Linreg__fit_intercept', 'Linreg__n_jobs', 'Linreg__normalize', 'Linreg__positive'])

In [31]:
gridsearchr.fit(trainr[featuresr], trainr['consumo_ps_x'])

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('Linreg', LinearRegression())]),
             param_grid={'Linreg__n_jobs': [1]})

In [32]:
gridsearchr.score(testr[featuresr], testr['consumo_ps_x'])

0.9796191307909677

Devo fare la stessa situa per la notte

In [33]:
BIGMERGED_night4

,precipitation_x,minTemperature_x,maxTemperature_x,cellId,timestamp_x,weekday_x,consumo_ps_x,precipitation_y,minTemperature_y,maxTemperature_y,consumo_ps_y,precipitation,minTemperature,maxTemperature,consumo_ps
0,0,-8.2,4.5,4074.0,1385679600,4.0,27.100023,0,-10.7,1.5,27.986804,0,-10.7,1.5,27.986804
1,0,-8.2,4.5,4191.0,1385679600,4.0,102.508784,0,-10.7,1.5,105.863129,0,-10.7,1.5,105.863129
2,0,-8.2,4.5,4307.0,1385679600,4.0,214.443664,0,-10.7,1.5,221.460798,0,-10.7,1.5,221.460798
3,0,-8.2,4.5,4308.0,1385679600,4.0,34.169595,0,-10.7,1.5,35.287710,0,-10.7,1.5,35.287710
4,0,-8.2,4.5,4423.0,1385679600,4.0,61.269618,0,-10.7,1.5,63.274514,0,-10.7,1.5,63.274514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43643,1,11.8,15.8,2147.0,1383865200,4.0,75.305624,0,3.9,17.8,76.111179,0,3.9,17.8,76.111179
43644,1,11.8,15.8,2148.0,1383865200,4.0,20.871615,0,3.9,17.8,21.614380,0,3.9,17.8,21.614380
43645,1,11.8,15.8,2261.0,1383865200,4.0,0.579142,0,3.9,17.8,0.604694,0,3.9,17.8,0.604694
43646,1,11.8,15.8,2262.0,1383865200,4.0,2.776123,0,3.9,17.8,2.863891,0,3.9,17.8,2.863891


In [34]:
trainr_n, testr_n = train_test_split(BIGMERGED_night4, test_size = 0.25)

In [35]:
gridsearchr.fit(trainr_n[featuresr], trainr_n['consumo_ps_x'])

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('Linreg', LinearRegression())]),
             param_grid={'Linreg__n_jobs': [1]})

In [36]:
gridsearchr.score(testr[featuresr], testr['consumo_ps_x'])

0.9794925984551462